# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

In [1]:
# 구글 드라이브 마운트, Colab을 이용하지 않는다면 패스해도 됩니다.
# from google.colab import drive
# drive.mount('/gdrive', force_remount=True)
# drive.mount('/content/drive')

Mounted at /gdrive
Mounted at /content/drive


In [2]:
# 구글 드라이브에 업로드된 대회 데이터를 압축 해제하고 로컬에 저장합니다.
# !tar -xvf drive/MyDrive/datasets_fin.tar > /dev/null

In [1]:
# 필요한 라이브러리를 설치합니다.
!pip install timm

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [2]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [3]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [4]:
# read data
train_df = pd.read_csv("data/train.csv")
train_df.head()

ID  target
0  002f99746285dfdd.jpg      16
1  008ccd231e1fea5d.jpg      10
2  008f5911bfda7695.jpg      10
3  009235e4c9c07af5.jpg       4
4  00b2f44967580c74.jpg      16

In [5]:
# Label data 수정
train_df.loc[train_df["ID"]=="8646f2c3280a4f49.jpg", ["target"]] = [3]

train_df.loc[train_df["ID"]=="0583254a73b48ece.jpg", ["target"]] = [6]
train_df.loc[train_df["ID"]=="38d1796b6ad99ddd.jpg", ["target"]] = [6]

train_df.loc[train_df["ID"]=="45f0d2dfc7e47c03.jpg", ["target"]] = [7]
train_df.loc[train_df["ID"]=="7100c5c67aecadc5.jpg", ["target"]] = [7]
train_df.loc[train_df["ID"]=="1ec14a14bbe633db.jpg", ["target"]] = [7]

train_df.loc[train_df["ID"]=="aec62dced7af97cd.jpg", ["target"]] = [14]
train_df.loc[train_df["ID"]=="c5182ab809478f12.jpg", ["target"]] = [14]

In [6]:
train_df.to_csv("data/train.csv", index=False)

In [7]:
# read data
train_df = pd.read_csv("data/train.csv")

In [8]:
print(train_df.loc[train_df["ID"]=="8646f2c3280a4f49.jpg", ["ID","target"]])
print(train_df.loc[train_df["ID"]=="0583254a73b48ece.jpg", ["ID","target"]])
print(train_df.loc[train_df["ID"]=="38d1796b6ad99ddd.jpg", ["ID","target"]])
print(train_df.loc[train_df["ID"]=="45f0d2dfc7e47c03.jpg", ["ID","target"]])
print(train_df.loc[train_df["ID"]=="1ec14a14bbe633db.jpg", ["ID","target"]])
print(train_df.loc[train_df["ID"]=="7100c5c67aecadc5.jpg", ["ID","target"]])
print(train_df.loc[train_df["ID"]=="aec62dced7af97cd.jpg", ["ID","target"]])
print(train_df.loc[train_df["ID"]=="c5182ab809478f12.jpg", ["ID","target"]])

                       ID  target
862  8646f2c3280a4f49.jpg       3
                      ID  target
38  0583254a73b48ece.jpg       6
                       ID  target
340  38d1796b6ad99ddd.jpg       6
                       ID  target
428  45f0d2dfc7e47c03.jpg       7
                       ID  target
192  1ec14a14bbe633db.jpg       7
                       ID  target
723  7100c5c67aecadc5.jpg       7
                        ID  target
1095  aec62dced7af97cd.jpg      14
                        ID  target
1237  c5182ab809478f12.jpg      14


In [9]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [10]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [13]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = 'data/'

# model config
model_name = 'resnet50' # 'resnet34' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 384 #256
crop_size = 336 # 추가
LR = 1e-3
EPOCHS = 5 #1
BATCH_SIZE = 64 # 32
num_workers = 0

## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [7]:
# # augmentation을 위한 transform 코드
# trn_transform = A.Compose([
#     # 이미지 크기 조정
#     A.Resize(height=img_size, width=img_size),
#     # images normalization
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
#     ToTensorV2(),
# ])

# # test image 변환을 위한 transform 코드
# tst_transform = A.Compose([
#     A.Resize(height=img_size, width=img_size),
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ToTensorV2(),
# ])

In [14]:
# 다양한 aug
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size), 
    A.RandomCrop(height=crop_size, width=crop_size),
    A.OneOf([
                A.HorizontalFlip(p=1),
                A.RandomRotate90(p=1),
                A.VerticalFlip(p=1)            
    ], p=1),
    A.OneOf([
                A.MotionBlur(p=1),
                A.OpticalDistortion(p=1),
                A.GaussNoise(p=1)                 
    ], p=1),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])


tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size), 
    A.RandomCrop(height=crop_size, width=crop_size),
    A.OneOf([
                A.HorizontalFlip(p=1),
                A.RandomRotate90(p=1),
                A.VerticalFlip(p=1)            
    ], p=1),
    A.OneOf([
                A.MotionBlur(p=1),
                A.OpticalDistortion(p=1),
                A.GaussNoise(p=1)                 
    ], p=1),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

In [15]:
# Dataset 정의
trn_dataset = ImageDataset(
    "data/train.csv",
    "data/train/",
    transform=trn_transform
)
tst_dataset = ImageDataset(
    "data/sample_submission.csv",
    "data/test/",
    transform=tst_transform
)
print(len(trn_dataset), len(tst_dataset))

1570 3140


In [16]:
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [ ]:
# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

In [18]:
for epoch in range(EPOCHS):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)

Loss: 1.2212: 100%|██████████| 25/25 [00:24<00:00,  1.04it/s]


train_loss: 2.1857
train_acc: 0.3924
train_f1: 0.4002
epoch: 0.0000



Loss: 0.2758: 100%|██████████| 25/25 [00:19<00:00,  1.29it/s]


train_loss: 0.7049
train_acc: 0.7713
train_f1: 0.7164
epoch: 1.0000



Loss: 0.4028: 100%|██████████| 25/25 [00:19<00:00,  1.30it/s]


train_loss: 0.3724
train_acc: 0.8688
train_f1: 0.8533
epoch: 2.0000



Loss: 0.1927: 100%|██████████| 25/25 [00:19<00:00,  1.30it/s]


train_loss: 0.2759
train_acc: 0.8949
train_f1: 0.8834
epoch: 3.0000



Loss: 0.3344: 100%|██████████| 25/25 [00:19<00:00,  1.29it/s]

train_loss: 0.2159
train_acc: 0.9204
train_f1: 0.9141
epoch: 4.0000



# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [19]:
preds_list = []

model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 50/50 [00:29<00:00,  1.69it/s]


In [20]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [21]:
sample_submission_df = pd.read_csv("data/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [22]:
pred_df.to_csv("pred_resnet50_1.csv", index=False)

In [16]:
pred_df.head(10)

ID  target
0  0008fdb22ddce0ce.jpg       2
1  00091bffdffd83de.jpg      12
2  00396fbc1f6cc21d.jpg       5
3  00471f8038d9c4b6.jpg       3
4  00901f504008d884.jpg       2
5  009b22decbc7220c.jpg      15
6  00b33e0ee6d59427.jpg       0
7  00bbdcfbbdb3e131.jpg       8
8  00c03047e0fbef40.jpg      15
9  00c0dabb63ca7a16.jpg      11

In [23]:
pred_df.head(10)

ID  target
0  0008fdb22ddce0ce.jpg       2
1  00091bffdffd83de.jpg      12
2  00396fbc1f6cc21d.jpg       5
3  00471f8038d9c4b6.jpg       0
4  00901f504008d884.jpg       2
5  009b22decbc7220c.jpg      15
6  00b33e0ee6d59427.jpg       0
7  00bbdcfbbdb3e131.jpg       8
8  00c03047e0fbef40.jpg      15
9  00c0dabb63ca7a16.jpg      11